<a href="https://colab.research.google.com/github/AvrodeepPal/LLMsPractice/blob/main/TokenLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 LLM Dataset Preprocessor: Gutenberg Edition

This script automates the process of downloading classic novels from [Project Gutenberg](https://www.gutenberg.org/), preprocessing the text, tokenizing sentences using a pretrained Hugging Face model, and decoding tokens back into readable text. This is ideal for creating clean datasets for LLM fine-tuning — including LoRA/PEFT workflows.

---

## 🚀 Features

- 📥 **Downloads** a random novel from Project Gutenberg
- ✂️ **Splits** it into clean, tokenized sentences
- 🧹 **Cleans** each sentence (lowercasing, punctuation removal, whitespace normalization)
- 🔢 **Tokenizes** sentences using `bert-base-uncased`
- 🧠 **Decodes** the tokenized IDs back into readable text
- ⚡ **Batch processing** and tqdm progress bars
- 🔧 Modular code structure for integration with larger pipelines

---

## 🛠️ Requirements

### Python Packages
Install dependencies via:

```bash
pip install -r requirements.txt
```

**`requirements.txt`**
```txt
transformers
nltk
tqdm
requests
```

---

## 📂 Output Files

| File            | Description                                   |
|-----------------|-----------------------------------------------|
| `novel.txt`     | One sentence per line from the downloaded novel |
| `encoded.txt`   | Tokenized version of each sentence (token IDs) |
| `decoded.txt`   | Reconstructed text from token IDs              |

---

## 📋 Step-by-Step Process

### ✅ Step 1: Download & Sentence Splitting
- Selects a random classic novel.
- Downloads full text.
- Removes Gutenberg headers/footers.
- Splits the novel into individual sentences using NLTK or regex fallback.
- Stores result in `novel.txt`.

### ✅ Step 2: Preprocessing & Tokenization
- Converts all text to lowercase.
- Strips punctuation and special characters (`a-z`, whitespace only).
- Tokenizes each sentence using `bert-base-uncased`.
- Stores tokenized IDs to `encoded.txt`.

### ✅ Step 3: Decoding Tokens to Text
- Converts token IDs back into readable English sentences.
- Uses the same tokenizer model for consistency.
- Stores output to `decoded.txt`.

---

## 📌 Usage

Run the full pipeline:
```bash
python your_script_name.py
```

You should see output like:
```bash
Resource punkt is already downloaded.
Selected book: Dracula (ID: 345)
Successfully processed 4,251 sentences to novel.txt
Successfully tokenized all sentences to encoded.txt
Successfully decoded all tokens to decoded.txt
Total execution time: 32.87 seconds
```

---

## ⚙️ Customization

- **Tokenizer**: Replace `"bert-base-uncased"` with any Hugging Face tokenizer.
- **Book List**: Add more books to the `gutenberg_books` list in `segment_1_download_and_split()`.
- **Cleaning**: Adjust regex cleaning rules to preserve punctuation or casing if needed.

---

## 🧠 Use Case: LoRA / PEFT Training

This pipeline is ideal for preparing clean sentence-level data for:
- Instruction tuning
- Language modeling (next token prediction)
- Encoder-decoder training (translation, summarization)
- Parameter-efficient fine-tuning using [LoRA](https://arxiv.org/abs/2106.09685) or [PEFT](https://github.com/huggingface/peft)

You can further export `encoded.txt` into `.jsonl`, `.parquet`, or `datasets.Dataset` formats.

---

## 📌 Notes

- Ensure you have internet access for downloading novels and Hugging Face tokenizers.
- Some books may use slightly different formats — the script tries alternative URLs automatically.
- Sentence splitting uses NLTK's `sent_tokenize()` and falls back to regex if resources are missing.

---

## 🧪 Sample Pipeline Snippet

```python
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
sentence = "It was the best of times, it was the worst of times."
cleaned = re.sub(r'[^a-z\s]', '', sentence.lower())
tokens = tokenizer(cleaned)["input_ids"]
decoded = tokenizer.decode(tokens)
```

---

## 🧵 Coming Soon (Ideas to Extend)

- ✅ Parallelization using `concurrent.futures` (for faster batch preprocessing)
- ✅ Save intermediate files as `.jsonl` or `datasets.Dataset`
- ✅ Add command-line flags (book selection, tokenizer choice, etc.)
- ✅ Word-level masking for pretraining
- ✅ Named Entity Recognition (NER) tagging

---

## 📄 License

MIT License ©️ 2025 Avrodeep Pal

---

In [ ]:
import random
import re
import requests
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer
from tqdm import tqdm
import time

# Download necessary NLTK data if not already present
# Make sure to download both 'punkt' and 'punkt_tab' resources
def ensure_nltk_resources():
    resources = ['punkt', 'punkt_tab']
    for resource in resources:
        try:
            nltk.data.find(f'tokenizers/{resource}')
            print(f"Resource {resource} is already downloaded.")
        except LookupError:
            print(f"Downloading {resource}...")
            nltk.download(resource)

def segment_1_download_and_split():
    """
    Downloads a random novel from Project Gutenberg and splits it into sentences.
    """
    # Some popular Gutenberg books with stable URLs
    gutenberg_books = [
        {"id": 1342, "title": "Pride and Prejudice"},
        {"id": 84, "title": "Frankenstein"},
        {"id": 11, "title": "Alice in Wonderland"},
        {"id": 1661, "title": "The Adventures of Sherlock Holmes"},
        {"id": 2701, "title": "Moby Dick"},
        {"id": 1400, "title": "Great Expectations"},
        {"id": 98, "title": "A Tale of Two Cities"},
        {"id": 1232, "title": "The Prince"},
        {"id": 74, "title": "The Adventures of Tom Sawyer"},
        {"id": 345, "title": "Dracula"}
    ]

    # Select a random book
    book = random.choice(gutenberg_books)
    print(f"Selected book: {book['title']} (ID: {book['id']})")

    # Construct the URL
    url = f"https://www.gutenberg.org/files/{book['id']}/{book['id']}-0.txt"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code != 200:
            # Try alternative URL format
            url = f"https://www.gutenberg.org/cache/epub/{book['id']}/pg{book['id']}.txt"
            response = requests.get(url, timeout=10)

        response.raise_for_status()  # Raise an exception for HTTP errors
        text = response.text
    except Exception as e:
        print(f"Error downloading the book: {e}")
        return False

    # Clean the text by removing Project Gutenberg headers and footers
    # This is a simplified approach - may need adjustment for different books
    start_marker = "*** START OF"
    end_marker = "*** END OF"

    if start_marker in text:
        text = text.split(start_marker)[1]
    if end_marker in text:
        text = text.split(end_marker)[0]

    # Alternative approach to sentence tokenization that doesn't require punkt_tab
    try:
        # Try using regular sent_tokenize first
        sentences = sent_tokenize(text)
    except LookupError:
        # Fallback to a simpler, less accurate sentence splitter
        print("NLTK sent_tokenize failed. Using simple regex splitter as fallback.")
        # Simple sentence splitter using regex
        sentences = re.split(r'(?<=[.!?])\s+', text)

    # Remove empty sentences and those with just whitespace
    sentences = [s.strip() for s in sentences if s.strip()]

    # Write to file, one sentence per line
    with open("novel.txt", "w", encoding="utf-8") as f:
        for sentence in sentences:
            f.write(sentence + "\n")

    print(f"Successfully processed {len(sentences)} sentences to novel.txt")
    return True

def segment_2_preprocess_and_tokenize():
    """
    Preprocesses each sentence and tokenizes it.
    """
    # Initialize tokenizer (use a smaller/faster model if speed is important)
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)

    # Process sentences in batches
    batch_size = 100
    sentences = []

    try:
        with open("novel.txt", "r", encoding="utf-8") as f:
            sentences = [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print("novel.txt not found! Run segment 1 first.")
        return False

    print(f"Processing {len(sentences)} sentences...")

    with open("encoded.txt", "w", encoding="utf-8") as f_out:
        # Process in batches with progress bar
        for i in tqdm(range(0, len(sentences), batch_size)):
            batch = sentences[i:i+batch_size]

            # Clean sentences
            cleaned_batch = []
            for sentence in batch:
                # Convert to lowercase
                s = sentence.lower()
                # Remove everything except a-z and whitespace
                s = re.sub(r'[^a-z\s]', '', s)
                # Replace multiple spaces with single space
                s = re.sub(r'\s+', ' ', s).strip()
                cleaned_batch.append(s)

            # Tokenize batch
            token_ids_batch = tokenizer(cleaned_batch, add_special_tokens=False)["input_ids"]

            # Write to file
            for token_ids in token_ids_batch:
                f_out.write(' '.join(map(str, token_ids)) + '\n')

    print(f"Successfully tokenized all sentences to encoded.txt")
    return True

def segment_3_decode():
    """
    Decodes token IDs back to text.
    """
    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)

    try:
        # Read token IDs
        with open("encoded.txt", "r", encoding="utf-8") as f:
            lines = [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print("encoded.txt not found! Run segment 2 first.")
        return False

    print(f"Decoding {len(lines)} tokenized sentences...")

    # Decode and write to file in batches
    with open("decoded.txt", "w", encoding="utf-8") as f_out:
        for i in tqdm(range(0, len(lines), 100)):
            batch = lines[i:i+100]

            for line in batch:
                # Convert space-separated string of IDs to list of integers
                try:
                    token_ids = list(map(int, line.split()))
                    # Decode back to text
                    decoded_text = tokenizer.decode(token_ids)
                    f_out.write(decoded_text + '\n')
                except ValueError:
                    # Skip lines that can't be properly converted
                    continue

    print(f"Successfully decoded all tokens to decoded.txt")
    return True

def main():
    start_time = time.time()

    # Ensure all required NLTK resources are available
    ensure_nltk_resources()

    print("SEGMENT 1: Downloading and sentence-splitting a novel...")
    if segment_1_download_and_split():
        print("\nSEGMENT 2: Preprocessing and tokenizing sentences...")
        if segment_2_preprocess_and_tokenize():
            print("\nSEGMENT 3: Decoding token IDs back to text...")
            segment_3_decode()

    total_time = time.time() - start_time
    print(f"\nTotal execution time: {total_time:.2f} seconds")

if __name__ == "__main__":
    main()

Resource punkt is already downloaded.


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


SEGMENT 1: Downloading and sentence-splitting a novel...
Selected book: Alice in Wonderland (ID: 11)
Successfully processed 984 sentences to novel.txt

SEGMENT 2: Preprocessing and tokenizing sentences...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Processing 2990 sentences...


100%|██████████| 30/30 [00:00<00:00, 43.91it/s]


Successfully tokenized all sentences to encoded.txt

SEGMENT 3: Decoding token IDs back to text...
Decoding 2981 tokenized sentences...


100%|██████████| 30/30 [00:05<00:00,  5.97it/s]

Successfully decoded all tokens to decoded.txt

Total execution time: 12.69 seconds


# 🧪 Tokenization Accuracy Evaluation

This script evaluates how accurately a tokenized and then decoded text reproduces the original cleaned version. It is useful for checking how much information is preserved after applying a tokenizer (like BERT) and then decoding it back to text.

---

## 📋 What It Does

- **Reads `cleaned.txt`** – the text preprocessed and cleaned before tokenization.
- **Reads `decoded.txt`** – the text obtained by decoding the token IDs.
- **Cleans both texts** again (lowercasing, removing punctuation).
- **Compares** them word by word using `difflib.SequenceMatcher`.
- **Calculates**:
  - Total words
  - Matched words
  - Overall tokenization-decoding accuracy (percentage)

---

## 🧩 Why Use It?

While tokenizers like BERT's are highly effective, they often split or normalize text differently during encoding and decoding (e.g., subwords, special tokens, etc.). This tool helps quantify how "lossy" that process is.

---

## 📁 Files Required

- `cleaned.txt`: Cleaned and preprocessed sentences (before tokenization).
- `decoded.txt`: Decoded sentences (from token IDs back to text).

These files are generated by the earlier stages in your pipeline:
- `cleaned.txt`: from `segment_2_preprocess_and_tokenize()`
- `decoded.txt`: from `segment_3_decode()`

---

## 🚀 How to Run

Make sure you have Python 3 and `tqdm` installed:

```bash
pip install tqdm
```

Then run:

```bash
python token_accuracy.py
```

You’ll see output like:

```
Found 10000 original sentences and 10000 decoded sentences
100%|████████████████████████████| 10000/10000
Tokenization Accuracy: 96.45%
Total words: 120000
Matched words: 115740
```

If the accuracy is low, the script provides reasons such as:
- Word splitting (e.g., "cannot" → "can not")
- Subword tokenization behavior
- Special characters/tokens during decoding

---

## 🔧 Optional Improvements

- Switch to character-level similarity metrics
- Log differences for manual review
- Add support for different tokenizers or models

---

## 📜 License

MIT License — free to use and modify.

---

In [ ]:
import re
import difflib
from tqdm import tqdm

def preprocess_text(text):
    """Clean text by converting to lowercase and removing non-alphabetic characters"""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def calculate_accuracy():
    print("Calculating tokenization accuracy...")

    # Read original sentences
    try:
        # use 'novel' for above code and 'cleaned' for below code
        with open("cleaned.txt", "r", encoding="utf-8") as f:
            original_lines = [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print("Error: novel.txt not found!")
        return

    # Read decoded sentences
    try:
        with open("decoded.txt", "r", encoding="utf-8") as f:
            decoded_lines = [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print("Error: decoded.txt not found!")
        return

    print(f"Found {len(original_lines)} original sentences and {len(decoded_lines)} decoded sentences")

    # Ensure we have similar lengths to compare
    min_length = min(len(original_lines), len(decoded_lines))
    if min_length == 0:
        print("Error: One or both files are empty!")
        return

    # Initialize metrics
    total_words = 0
    matched_words = 0

    # Compare each line with sequence matching
    for i in tqdm(range(min_length)):
        # Preprocess both lines
        orig_clean = preprocess_text(original_lines[i])
        decoded_clean = preprocess_text(decoded_lines[i])

        # Split into words
        orig_words = orig_clean.split()
        decoded_words = decoded_clean.split()

        # Use sequence matcher to find best alignment
        matcher = difflib.SequenceMatcher(None, orig_words, decoded_words)

        # Count matched words
        for match in matcher.get_matching_blocks():
            i_start, j_start, match_len = match
            if match_len > 0:  # Skip the sentinel at the end
                matched_words += match_len

        # Add to total word count
        total_words += len(orig_words)

    # Calculate accuracy
    if total_words > 0:
        accuracy = (matched_words / total_words) * 100
        print(f"\nTokenization Accuracy: {accuracy:.2f}%")
        print(f"Total words: {total_words}")
        print(f"Matched words: {matched_words}")

        # Detailed breakdown
        if accuracy < 95:
            print("\nReasons for lower accuracy may include:")
            print("- Word splitting differences (e.g., 'cannot' → 'can not')")
            print("- Tokenizer normalization (e.g., whitespace handling)")
            print("- Special tokens or subword tokenization")
    else:
        print("No words to compare!")

if __name__ == "__main__":
    calculate_accuracy()

Calculating tokenization accuracy...
Found 8803 original sentences and 8803 decoded sentences


100%|██████████| 8803/8803 [00:00<00:00, 56395.31it/s]


Tokenization Accuracy: 100.00%
Total words: 74979
Matched words: 74979


# 📚 Novel Sentence Tokenization & Token ID Pipeline

This Python project demonstrates a complete pipeline for:

1. Downloading a random novel from [Project Gutenberg](https://www.gutenberg.org/),
2. Splitting it into sentences,
3. Cleaning and tokenizing the text using a pretrained BERT tokenizer,
4. Saving token IDs,
5. Decoding token IDs back to text,
6. Verifying decoding accuracy.

Perfect for tasks like:
- Building datasets for language modeling
- Experimenting with tokenizers
- NLP pipeline demonstrations

---

## 🛠️ Features

- Downloads 1 out of 10 popular public domain novels
- Cleans and splits the book into well-formed sentences
- Tokenizes using `bert-base-uncased` (via Hugging Face Transformers)
- Parallel processing for fast tokenization & decoding
- Cleaned text, token IDs, and decoded text all saved to file
- Automatic verification of tokenization–decoding similarity

---

## 📁 File Outputs

| File Name     | Description                                  |
|---------------|----------------------------------------------|
| `novel.txt`   | Original sentences from the downloaded novel |
| `cleaned.txt` | Cleaned, normalized sentences                |
| `encoded.txt` | BERT token IDs per sentence (space-separated)|
| `decoded.txt` | Decoded text from token IDs                  |

---

## 🧩 Segments Breakdown

### 🔹 Segment 1: Download & Split
- Randomly selects a novel from a curated list
- Downloads and cleans Project Gutenberg text
- Splits into individual sentences using NLTK or regex fallback

### 🔹 Segment 2: Preprocess & Tokenize
- Cleans each sentence (lowercase, remove punctuation)
- Tokenizes using `bert-base-uncased`
- Uses `ThreadPoolExecutor` for parallel processing
- Saves `cleaned.txt` and `encoded.txt`

### 🔹 Segment 3: Decode
- Decodes `encoded.txt` (token IDs) back to natural text
- Saves to `decoded.txt`
- Compares with `cleaned.txt` to verify decoding accuracy

---

## 🚀 Getting Started

### 1. Clone the repo

```bash
git clone https://github.com/your-username/novel-tokenization-pipeline.git
cd novel-tokenization-pipeline
```

### 2. Install dependencies

```bash
pip install transformers nltk tqdm requests
```

### 3. Run the full pipeline

```bash
python your_script_name.py
```

---

## 🧠 Example Output

```bash
Selected book: Frankenstein (ID: 84)
Successfully processed 3,542 sentences to novel.txt
Processing 3,542 sentences...
Successfully tokenized all sentences to encoded.txt and saved cleaned text to cleaned.txt
Decoding 3,542 tokenized sentences...
Decoding verification: 95.72% similarity with cleaned text
Successfully decoded all tokens to decoded.txt
```

---

## 🧪 Project Goals

This project serves as a stepping stone for:

- Data preparation for LLM training
- Testing tokenizer-decoder fidelity
- Building your own text-to-token-to-text pipeline

---

## 📌 Notes

- If `nltk.punkt` is not installed, it will be auto-downloaded.
- Regex fallback is used when `sent_tokenize` fails.
- You can swap in a different tokenizer/model in Segment 2 easily.

---

## 🧑‍💻 Author

Made with ❤️ by [Avrodeep Pal]

---


In [ ]:
import random
import re
import requests
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer
from tqdm import tqdm
import time
import concurrent.futures
from functools import partial

# Download necessary NLTK data if not already present
def ensure_nltk_resources():
    resources = ['punkt', 'punkt_tab']
    for resource in resources:
        try:
            nltk.data.find(f'tokenizers/{resource}')
            print(f"Resource {resource} is already downloaded.")
        except LookupError:
            print(f"Downloading {resource}...")
            nltk.download(resource)

def segment_1_download_and_split():
    """
    Downloads a random novel from Project Gutenberg and splits it into sentences.
    """
    # Some popular Gutenberg books with stable URLs
    gutenberg_books = [
        {"id": 1342, "title": "Pride and Prejudice"},
        {"id": 84, "title": "Frankenstein"},
        {"id": 11, "title": "Alice in Wonderland"},
        {"id": 1661, "title": "The Adventures of Sherlock Holmes"},
        {"id": 2701, "title": "Moby Dick"},
        {"id": 1400, "title": "Great Expectations"},
        {"id": 98, "title": "A Tale of Two Cities"},
        {"id": 1232, "title": "The Prince"},
        {"id": 74, "title": "The Adventures of Tom Sawyer"},
        {"id": 345, "title": "Dracula"}
    ]

    # Select a random book
    book = random.choice(gutenberg_books)
    print(f"Selected book: {book['title']} (ID: {book['id']})")

    # Construct the URL
    url = f"https://www.gutenberg.org/files/{book['id']}/{book['id']}-0.txt"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code != 200:
            # Try alternative URL format
            url = f"https://www.gutenberg.org/cache/epub/{book['id']}/pg{book['id']}.txt"
            response = requests.get(url, timeout=10)

        response.raise_for_status()  # Raise an exception for HTTP errors
        text = response.text
    except Exception as e:
        print(f"Error downloading the book: {e}")
        return False

    # Clean the text by removing Project Gutenberg headers and footers
    start_marker = "*** START OF"
    end_marker = "*** END OF"

    if start_marker in text:
        text = text.split(start_marker)[1]
    if end_marker in text:
        text = text.split(end_marker)[0]

    # Alternative approach to sentence tokenization that doesn't require punkt_tab
    try:
        # Try using regular sent_tokenize first
        sentences = sent_tokenize(text)
    except LookupError:
        # Fallback to a simpler, less accurate sentence splitter
        print("NLTK sent_tokenize failed. Using simple regex splitter as fallback.")
        # Simple sentence splitter using regex
        sentences = re.split(r'(?<=[.!?])\s+', text)

    # Remove empty sentences and those with just whitespace
    sentences = [s.strip() for s in sentences if s.strip()]

    # Write to file, one sentence per line
    with open("novel.txt", "w", encoding="utf-8") as f:
        for sentence in sentences:
            f.write(sentence + "\n")

    print(f"Successfully processed {len(sentences)} sentences to novel.txt")
    return True

def clean_sentence(sentence):
    """Helper function to clean individual sentences"""
    # Convert to lowercase
    s = sentence.lower()
    # Remove everything except a-z and whitespace
    s = re.sub(r'[^a-z\s]', '', s)
    # Replace multiple spaces with single space
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def process_batch(batch, tokenizer):
    """Process a batch of sentences with tokenizer"""
    # Clean sentences
    cleaned_batch = [clean_sentence(sentence) for sentence in batch]

    # Tokenize batch
    token_ids_batch = tokenizer(cleaned_batch, add_special_tokens=False)["input_ids"]

    # Return results
    return list(zip(cleaned_batch, [' '.join(map(str, token_ids)) for token_ids in token_ids_batch]))

def segment_2_preprocess_and_tokenize():
    """
    Preprocesses each sentence and tokenizes it using parallel processing.
    """
    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)

    # Read sentences
    try:
        with open("novel.txt", "r", encoding="utf-8") as f:
            sentences = [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print("novel.txt not found! Run segment 1 first.")
        return False

    print(f"Processing {len(sentences)} sentences...")

    # Determine optimal batch size and number of workers
    batch_size = 100
    max_workers = min(32, (len(sentences) // batch_size) + 1)
    batches = [sentences[i:i+batch_size] for i in range(0, len(sentences), batch_size)]

    # Process batches in parallel
    results = []
    process_func = partial(process_batch, tokenizer=tokenizer)

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all batches to the executor
        future_to_batch = {executor.submit(process_func, batch): i for i, batch in enumerate(batches)}

        # Process results as they complete
        for future in tqdm(concurrent.futures.as_completed(future_to_batch), total=len(batches)):
            batch_results = future.result()
            results.extend(batch_results)

    # Sort results based on the order they were submitted
    # This is necessary because parallel processing might return results out of order
    results.sort(key=lambda x: sentences.index(x[0]) if x[0] in sentences else float('inf'))

    # Write results to file - both original cleaned text and tokenized IDs
    with open("cleaned.txt", "w", encoding="utf-8") as f_cleaned:
        with open("encoded.txt", "w", encoding="utf-8") as f_encoded:
            for cleaned_text, encoded_ids in results:
                f_cleaned.write(cleaned_text + '\n')
                f_encoded.write(encoded_ids + '\n')

    print(f"Successfully tokenized all sentences to encoded.txt and saved cleaned text to cleaned.txt")
    return True

def decode_batch(batch, tokenizer):
    """Decode a batch of tokenized sentences"""
    results = []
    for line in batch:
        try:
            token_ids = list(map(int, line.split()))
            decoded_text = tokenizer.decode(token_ids)
            results.append(decoded_text)
        except ValueError:
            # Skip lines that can't be properly converted
            continue
    return results

def segment_3_decode():
    """
    Decodes token IDs back to text using parallel processing.
    """
    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)

    try:
        # Read token IDs
        with open("encoded.txt", "r", encoding="utf-8") as f:
            lines = [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print("encoded.txt not found! Run segment 2 first.")
        return False

    print(f"Decoding {len(lines)} tokenized sentences...")

    # Determine batch size and number of workers
    batch_size = 100
    max_workers = min(32, (len(lines) // batch_size) + 1)
    batches = [lines[i:i+batch_size] for i in range(0, len(lines), batch_size)]

    # Process batches in parallel
    results = []
    decode_func = partial(decode_batch, tokenizer=tokenizer)

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all batches to the executor
        future_to_batch = {executor.submit(decode_func, batch): i for i, batch in enumerate(batches)}

        # Process results as they complete with tqdm for progress tracking
        for i, future in enumerate(tqdm(concurrent.futures.as_completed(future_to_batch), total=len(batches))):
            batch_idx = list(future_to_batch.values())[list(future_to_batch.keys()).index(future)]
            batch_results = future.result()
            # Store results with their batch index for proper ordering
            results.append((batch_idx, batch_results))

    # Sort results by batch index to maintain original order
    results.sort(key=lambda x: x[0])

    # Flatten the sorted results
    all_decoded = []
    for _, batch_results in results:
        all_decoded.extend(batch_results)

    # Write results to file
    with open("decoded.txt", "w", encoding="utf-8") as f_out:
        for text in all_decoded:
            f_out.write(text + '\n')

    # Verify decoding by comparing with cleaned text
    try:
        with open("cleaned.txt", "r", encoding="utf-8") as f_cleaned:
            cleaned_lines = [line.strip() for line in f_cleaned if line.strip()]

        # Check if counts match
        if len(cleaned_lines) != len(all_decoded):
            print(f"Warning: Number of cleaned lines ({len(cleaned_lines)}) doesn't match decoded lines ({len(all_decoded)})")
        else:
            # Calculate simple similarity score
            similarity_count = sum(1 for i in range(min(len(cleaned_lines), len(all_decoded)))
                                if cleaned_lines[i].lower() == all_decoded[i].lower())
            similarity_percentage = (similarity_count / len(cleaned_lines)) * 100
            print(f"Decoding verification: {similarity_percentage:.2f}% similarity with cleaned text")
    except FileNotFoundError:
        print("cleaned.txt not found. Skipping verification.")

    print(f"Successfully decoded all tokens to decoded.txt")
    return True

def main():
    start_time = time.time()

    # Ensure all required NLTK resources are available
    ensure_nltk_resources()

    print("SEGMENT 1: Downloading and sentence-splitting a novel...")
    if segment_1_download_and_split():
        print("\nSEGMENT 2: Preprocessing and tokenizing sentences...")
        if segment_2_preprocess_and_tokenize():
            print("\nSEGMENT 3: Decoding token IDs back to text...")
            segment_3_decode()

    total_time = time.time() - start_time
    print(f"\nTotal execution time: {total_time:.2f} seconds")

if __name__ == "__main__":
    main()

Resource punkt is already downloaded.
Resource punkt_tab is already downloaded.
SEGMENT 1: Downloading and sentence-splitting a novel...
Selected book: Frankenstein (ID: 84)
Successfully processed 3075 sentences to novel.txt

SEGMENT 2: Preprocessing and tokenizing sentences...
Processing 8803 sentences...


100%|██████████| 89/89 [00:00<00:00, 95618.10it/s]


Successfully tokenized all sentences to encoded.txt and saved cleaned text to cleaned.txt

SEGMENT 3: Decoding token IDs back to text...
Decoding 8803 tokenized sentences...


100%|██████████| 89/89 [00:00<00:00, 122915.07it/s]

Decoding verification: 100.00% similarity with cleaned text
Successfully decoded all tokens to decoded.txt

Total execution time: 3.61 seconds


In the optimized code I provided, cleaned.txt is a new intermediate file that was added to help track and debug the text processing pipeline. Let me explain its purpose and content:
What is cleaned.txt?
cleaned.txt is a file that contains the "cleaned" version of each sentence from the original novel before tokenization occurs. By "cleaned," I mean the text after all the preprocessing steps have been applied but before it's converted to token IDs.
What preprocessing is applied to create cleaned.txt?
Each sentence undergoes the following transformations:

Lowercase conversion: All text is converted to lowercase
Character filtering: Everything except lowercase letters (a-z) and spaces is removed
Space normalization: Multiple spaces are condensed into a single space, and leading/trailing spaces are trimmed

For example, a sentence like:
"Hello, world! How are you doing today?"
Would become:
"hello world how are you doing today"
Why was cleaned.txt added?
Adding this file serves several important purposes:

Debugging aid: It allows you to see exactly what text is being tokenized, making it easier to identify issues in the process
Verification reference: The decoded output should match this cleaned text (if the tokenization and decoding are working correctly)
Order preservation: By keeping this file, we can verify that the order of sentences is preserved throughout the parallel processing pipeline
Quality check: The code now includes a verification step that compares the decoded text with the cleaned text and reports a similarity percentage

How is cleaned.txt used in the process?

During segment_2_preprocess_and_tokenize(), each sentence is cleaned and its cleaned version is saved alongside its token IDs
Both versions are written to their respective files (cleaned.txt and encoded.txt), maintaining the same order
In segment_3_decode(), after decoding all token IDs back to text, the code compares the decoded text with the cleaned text to verify accuracy

This intermediate file helps ensure that what goes into the tokenization process matches what comes out of the decoding process, providing a way to validate that the entire pipeline is working correctly. If there's a significant mismatch between cleaned.txt and decoded.txt, it would indicate a problem in the tokenization/detokenization process.